In [ ]:
!pip install geopandas
!pip install tqdm

In [1]:
import os
import boto3
import rasterio
import geopandas as gpd

from copy import copy
from shapely.geometry import box
from tqdm import tqdm_notebook as tqdm

Buffer turbines to create polygons

In [2]:
wind_turbines = gpd.read_file('uswtdb_v1_3_20190107.geojson')

In [3]:
selected_turbines = wind_turbines[wind_turbines.t_state.isin(['IA', 'OK', 'TX'])]

In [4]:
selected_turbines_poly = copy(selected_turbines)
selected_turbines_poly.geometry = selected_turbines.to_crs(epsg=3857).buffer(50)

In [5]:
s3 = boto3.client('s3')

In [6]:
bucket='rasterfoundry-production-data-us-east-1'

In [12]:
states = [('ia', '2017'), ('ok', '2017'), ('tx', '2016')]

In [13]:
def create_labels_for_scene(tif_uri, state, turbines):
    with rasterio.open(uri) as d:
        data = d.bounds
        crs = int(d.crs['init'].split(':')[1])
    
    bbox = box(data[0], data[1], data[2], data[3])
    turbines = turbines.to_crs(epsg=crs)
    tmp = turbines[turbines.intersects(bbox)]
    tmp = tmp.to_crs({'init': 'epsg:4326'})
    
    if len(tmp) > 0:
        geojson_file = os.path.basename(tif_uri).replace('.tif', '.geojson')
        outfile = os.path.join('output/', geojson_file)
        tmp.to_file(driver='GeoJSON', filename=outfile)
        
        geojson_bucket = 'raster-vision-wind-turbines'
        filename = 'labels/{}/{}'.format(state, geojson_file)
        s3.upload_file(outfile, geojson_bucket, filename)

In [14]:
for state, year in tqdm(states):
    prefix_dir = 'naip-visualization/{}/{}/100cm/rgb/'.format(state, year)
    all_directories = s3.list_objects(Bucket=bucket, Prefix=prefix_dir, Delimiter='/')
    prefixes = [x['Prefix'] for x in all_directories['CommonPrefixes']]
    
    dir_files = []
    
    pbar = tqdm(prefixes)
    description = 'Generating list of tifs in {}:'.format(state)
    pbar.set_description(description)
    for prefix in pbar:
        tifs = s3.list_objects(Bucket=bucket, Prefix=prefix)
        dir_files += [x['Key'] for x in tifs['Contents']]
    
    pbar = tqdm(dir_files)
    description = 'Checking for wind turbines in {}: '.format(state)
    pbar.set_description(description)
    for d in pbar:
        uri = 's3://{}/{}'.format(bucket, d)
        create_labels_for_scene(uri, state, selected_turbines_poly)

/usr/local/lib/python3.5/dist-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


KeyboardInterrupt: 